In [2]:
# import tensorflow as tf

In [3]:
#build params (Hyperparameters)

In [17]:
config = {}
EPOCHS = 15
BATCH_SIZE = 32
LR = 0.001
DECAY_LR = 0.9
OPTM = "Adam"
DROPOUT = 0.3
EPOCHS_NO_IMPROVEMENT = 4
HIDDEN_DIM = 256
DATA_FILEPATH = "data_folder/data.txt"
GLOVE_FILEPATH = "glove/"
VOCAB_FILENAME = "data_folder/vocabs.txt"
TAG_FILENAME = "data_folder/tags.txt"



In [5]:
#build files from data

In [6]:
def data_batches(data,batch_size=32):
    """Args >> data=tuple(sentence,tag)
    returns data in batches"""
    steps = len(data)//batch_size
    x_batch, y_batch = [], []
    for (x,y) in data:
        if len(x_batch) == batch_size:
            yield x_batch, y_batch
            x_batch, y_batch = [], []
        if type(x[0])==tuple:
            x = zip(*x)
            x_batch += [x]
            y_batch += [y]
    if len(x_batch) != 0:
        yield x_batch, y_batch

In [7]:
UNK = "$UNK$"
NUM = "$NUM$"
NONE = "O"

In [10]:
def get_vocabs(datasets):
    """Building vocabs present in datasets"""
    print("Building Vocab ....")
    vocab_words = set()
    vocab_tags = set()
    for dataset in datasets:
        for words, tags in dataset:
            vocab_words.update(words)
            vocab_tags.update(tags)
    print("-done vocab words {s}--- vocab_tags {d}".format(s=vocab_words,d=vocab_tags))
    return vocab_words, vocab_tags

In [11]:
def get_glove_vocab(path):
    """Building vocabs present in glove"""
    print("Building Vocab..glove...")
    vocab = set()
    with open(path,'r') as f:
        for line in f:
            word = line.strip().split(' ')[0]
            vocab.add(word)
    print("done. {} tokens in glove".format(len(vocab)))
    return vocab

In [12]:
def final_vocab(vocab_words,vocab_glove,**args):
    """Combine word vocabs >> glove_vocabs+dataset_vocabs"""
    vocab = vocab_words & vocab_glove
    for word in args:
        vocab.add(word)
    return vocab
    

In [13]:
def build_vocab():
    "Write all vocabs line by line in a file"
    with open(filename,'w'):
        for i, word in enumerate(voacb):
            if i != len(vocab)-1:
                f.write("{}\n".format(word))
            else:
                f.write(word)
    print("written {s} tokens in {d}".format(s=len(vocab), d=filename))


In [14]:
def load_vocab(filename):
    """Assign id to each word in vocab
        returns dictionary"""
    d = {}
    with open(filename) as f:
        for idx, word in enumerate(f):
            word = word.strip()
            d[word] = idx
    return d

In [15]:
def export_glove_vectors(vocab, glove_filename, filename, dim):
    """Bulding compressed file of vectors of words
    that are present in dataset"""
    embeddings = np.zeroes(len(vocab),dim)
    with open(glove_filename,'r',encoding='utf-8') as f:
        for line in f:
            line = line.strip().split(' ')
            word = line[0]
            if word in vocab:
                embedding = [float(x) for x in line[1:]]
                word_idx = vocab[word]
                embeddings[word_idx] = np.asarray(embedding)
    np.savez_compressed(filename, embeddings=embeddings)

def get_glove_vectors(filename):
    """Loads the saved numpy file (Embeddings)"""
    with np.load(filename) as data:
        return data['embeddings']